<a href="https://colab.research.google.com/github/marcmontb/VC-sourcing-engine/blob/main/Github_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask pyngrok pygithub transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 18.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

Celda 2: Importar Bibliotecas y Configuración Inicial


In [ ]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from github import Github, GithubException
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Configura tu authtoken de GitHub y ngrok
GITHUB_TOKEN = ""
ngrok.set_auth_token("")

# Inicializar la app Flask
app = Flask(__name__)


Celda 3: Cargar el Modelo y Definir Funciones de Embeddings y Similitud


In [ ]:
# Cargar el modelo y el tokenizador de transformers
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Celda 4: Configurar la Ruta Principal de la Aplicación Flask


In [ ]:
@app.route('/', methods=['GET', 'POST'])
def home():
    startups_info = []
    if request.method == 'POST':
        # Obtener filtros del formulario
        language = request.form.get('language')
        stars = request.form.get('stars')
        forks = request.form.get('forks')
        created_after = request.form.get('created_after')
        min_contributors = request.form.get('min_contributors')
        keywords = request.form.get('keywords')

        # Obtener embedding para las palabras clave
        keyword_embedding = get_embedding(keywords)

        # Construir la consulta
        query = []
        if language:
            query.append(f"language:{language}")
        if stars:
            query.append(f"stars:>{stars}")
        if forks:
            query.append(f"forks:>{forks}")
        if created_after:
            query.append(f"created:>{created_after}")

        query = " ".join(query)

        try:
            # Autenticar en GitHub
            g = Github(GITHUB_TOKEN)
            # Buscar repositorios en GitHub
            result = g.search_repositories(query=query)

            # Filtrar proyectos que contengan subpalabras y parezcan startups
            startups = []
            for repo in result:
                description = repo.description or ""
                readme_content = repo.get_readme().decoded_content.decode() if repo.get_readme() else ""
                combined_text = description + " " + readme_content

                # Obtener embedding del texto combinado
                text_embedding = get_embedding(combined_text)
                similarity = cosine_similarity(keyword_embedding, text_embedding)

                if similarity > 0.5:  # Umbral de similitud
                    if repo.open_issues_count > 0:
                        contributors = repo.get_contributors()
                        if len(contributors) > int(min_contributors):
                            # Obtener información adicional
                            license = repo.get_license().license.name if repo.get_license() else 'No License'
                            commits = repo.get_commits()
                            last_commit = commits[0].commit.author.date if commits.totalCount > 0 else 'No commits'
                            startups.append((repo.full_name, repo.description, repo.html_url, license, last_commit, combined_text[:200]))

            # Preparar la información para mostrar
            startups_info = startups
        except GithubException as e:
            print(f"Error al autenticar o buscar en GitHub: {e}")

    # HTML con el formulario y la lista de resultados
    html = """
    <h1>Buscar Startups en GitHub</h1>
    <form method="post">
        Lenguaje de Programación: <input type="text" name="language"><br>
        Estrellas Mínimas: <input type="number" name="stars"><br>
        Forks Mínimos: <input type="number" name="forks"><br>
        Creado Después de (YYYY-MM-DD): <input type="date" name="created_after"><br>
        Contribuidores Mínimos: <input type="number" name="min_contributors"><br>
        Palabras Clave: <input type="text" name="keywords"><br>
        <input type="submit" value="Buscar">
    </form>
    <h2>Resultados:</h2>
    <ul>
    {% for name, desc, url, license, last_commit, text in startups %}
        <li>
            <strong><a href="{{ url }}" target="_blank">{{ name }}</a></strong>: {{ desc }}<br>
            <strong>License:</strong> {{ license }}<br>
            <strong>Last Commit:</strong> {{ last_commit }}<br>
            <strong>Text:</strong> {{ text }}...
        </li>
    {% endfor %}
    </ul>
    """
    return render_template_string(html, startups=startups_info)


In [ ]:
# Ejecutar la app Flask con ngrok
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
app.run()


 * ngrok tunnel "NgrokTunnel: "https://7766-34-106-87-201.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 17:30:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 17:30:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 17:31:13] "POST / HTTP/1.1" 200 -


Error al autenticar o buscar en GitHub: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/contents#get-a-repository-readme", "status": "404"}


INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 17:31:45] "POST / HTTP/1.1" 200 -


Error al autenticar o buscar en GitHub: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/contents#get-a-repository-readme", "status": "404"}
